#### Na kreslenie budeme používať modul `plotly`, konkrétne `plotly.express`, čo aj vám odporúčame.

In [ ]:
import polars as pl
import plotly.express as px
import numpy as np  # pre vektorove operacie

In [ ]:
# nacitame vycistenu frejmu, len stlpce, co pouzijeme v tomto NB
cols = ['pick_dt', 'passengers', 'fare']
df = pl.scan_parquet('data/nyc_taxi310k.parq')   # pozn. o lazy frame
df = df.select(cols).collect()                   # collect sposobi, ze sa z lazy stane normalna frejma

In [ ]:
df.min(), df.max() # min. a max. vo vsetkych stlpcov

### Všetky dáta sú z januára 2015. 
#### Zoberme nástupy (pick). Chceme grafy po jednotlivých dňoch. Počet pasažierov, počet jázd, zarobené peniaze.

In [ ]:
# pocet cestujucich, najskor bez 'alias'
(df.group_by(pl.col('pick_dt').dt.day())
            .agg(pl.col('passengers').sum()))
           
df_days = (df.group_by(pl.col('pick_dt').dt.day().alias('pick_day'))
             .agg(pl.col('passengers').sum().alias('pass_count')))

In [ ]:
# pred a po alias ukazat
df_days.head()

### Nakreslíme stĺpcový graf, na osi x budú dni (1, 2, .... 31. jan.), na osi y počty cestujúcich

In [ ]:
pass_plot = px.bar(df_days, x='pick_day', y='pass_count', labels={'pick_day':'Dni', 'pass_count': 'Cestujúci'})
# dni v mesiaci chceme pre kazdy den
pass_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 32))));

In [ ]:
pass_plot
# ukazat hover a nastroje plotly

## Čo za katastrófa sa stala 27. januára? Strašný pokles oproti iným dňom.
#### Vidíme nejakú (približnú) periodicitu v dátach?
### Podobné grafy by sme chceli pre počty jázd a pre denné zárobky.

In [ ]:
# zrobime frejmu, kde budu vsetky tie tri veliciny agregovane podla dni - ako hore, len zoznam troch stlpcov namiesto jedneho

# df_days = 
(df.group_by(pl.col('pick_dt').dt.day().alias('pick_day'))
             .agg([pl.col('passengers').sum().alias('pass_count'),
                   pl.col('fare').sum().alias('day_fares'), 
                   pl.col('fare').count().alias('fares_count')])).sort(by='pick_day')

# namiesto alias mozno dat nove nazvy stlpcov
df_days = (df.group_by(pl.col('pick_dt').dt.day().alias('pick_day'))
             .agg(pass_count=pl.col('passengers').sum(),
                  day_fares=pl.col('fare').sum(), 
                  fares_count= pl.col('fare').count())).sort(by='pick_day')

In [ ]:
# ake asi budu skutocne trzby, ked my mame vzorku 310 000 zapisov z 12 milionov
df_days.head()

In [ ]:
# mozeme nakreslit kombinovany graf pre pocet jazd a pocet cestujucich (malo by to byt zhruba radovo rovnake)

pass_fares_plot = px.bar(df_days, x='pick_day', y=['pass_count', 'fares_count'], barmode='group',)
                         # labels={'pick_day':'Dni', 'value': 'Hodnoty', 'variable': 'Premenná'})
# zobrazenie sa nemiesa s jeho rozpolozenim (layout)    
# pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 32))))
pass_fares_plot

### Grafy pre tie isté veličiny, no podľa hodín - to isté, len grupovanie podľa hodín
Jediná funkcia pre datafrejmu a aj pre graf.

In [ ]:
def monthly_frame(frm, day=True):      # nazvy stplcov v datafrejme mozu byt aj po slovensky :-)
    groupped = frm.group_by(pl.col('pick_dt').dt.day()) if day else\
               frm.group_by(pl.col('pick_dt').dt.hour())         
    column = 'pick_day' if day else 'pick_hour'
    df_month = groupped.agg([pl.col('fare').sum().alias('Platby'), 
                             pl.col('passengers').sum().alias('Cestujúci'),
                             pl.col('fare').count().alias('Jazdy')]).sort(by='pick_dt')
    df_month = df_month.rename({'pick_dt': column})   # pick_dt sa tu nehodi, bude pick_day alebo pick_hour
    return df_month

# V aplikacii budeme mat radiobox s volbami ['Podľa dní','Podľa hodín', 'Dni v týždni (nástupy)']

def monthly_plot(dhc):
    day = (dhc == 'Podľa dní')
    xcol = 'pick_day' if day else 'pick_hour'
    mframe = monthly_frame(df, day)
    xcol = 'pick_day' if day else 'pick_hour'
    xlabel = {'pick_day': 'Deň', 'pick_hour': 'Hodina'}
    xticks = {'pick_day': list(range(1, 32)), 'pick_hour': list(range(24))}
    pass_fares_plot = px.bar(mframe, x=xcol, y=['Cestujúci', 'Jazdy'], barmode='group',
                             labels={xcol: xlabel[xcol], 'value': 'Hodnoty', 'variable': 'Premenná'})
    pass_fares_plot.update_layout(xaxis=dict(tickmode='array', tickvals=xticks[xcol]))
    return pass_fares_plot

In [ ]:
monthly_plot('Podľa dní')

In [ ]:
monthly_plot('Podľa nední') #  facina, samozrejme

#### Hore vyrobene dve funkcie hodime do suboru `data_funkcie.py`. V dalsich NB ich budeme odtial importovat.
#### Nebolo by odveci, mať graf aj pre cestujúcich, jazdy podľa dní v týždni.

In [ ]:
df_weekday = (df.group_by(pl.col('pick_dt').dt.weekday())
                .agg(pl.col('passengers').sum().alias('pass_count'))
                .sort(by='pick_dt'))
df_weekday = df_weekday.rename({'pick_dt': 'pick_day'})

In [ ]:
df_weekday  # je to len 7 poloziek, netreba head. z dokumentacie polars - 1 je pondelok
# df_weekday['pass_count'].sum() # hruba kontrola

In [ ]:
# nie je to v poriadku, nie kazdy den v tyzdni je v januari 2015 rovnaky pocet raz
# chceme pocty za jeden pondelok, utorok, ...
# klucove - aky den bol 1. januara 2015?
# datetime ma funkciu weekday 
from datetime import date
print(date(2015, 1, 1).weekday()) # 0 je tu pondelok, ... 6 je nedela - nie ako v polars

In [ ]:
# teda 1. jan. 2015 bol stvrtok a pocty dni v januari su
# 1, 8, 15, 22, 29 - stvrtky, 30-pia, 31-so bude po 5, ostatne dni po 4
pocty = np.array([4, 4, 4, 5, 5, 5, 4])  # preco nie len list?
df_weekday = df_weekday.with_columns(pl.col('pass_count') / pocty)  # bez alias meno stlpca ostane
df_weekday

In [ ]:
graf = px.bar(df_weekday, x='pick_day', y='pass_count', barmode='group', width=750, height=400)
xtext = ['Pondelok', 'Utorok', 'Streda', 'Štvrtok', 'Piatok', 'Sobota', 'Nedeľa']
graf.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 8)), title='Deň v týždni',
                   ticktext=xtext, tickangle=0), yaxis=dict(title="Priem. počet cestujúcich"))
# zopakujte s count() namiesto sum() - bude priem. pocet jazd

In [ ]:
# zosumarizujeme, vyrobime funkcie week_plot, view_month_week a hodime ich do data_funkcie.py
def week_plot(frm):
    df_weekday = (frm.group_by(pl.col('pick_dt').dt.weekday())
                     .agg(pl.col('passengers').count().alias('pass_count'))  # co keby sum namiesto count?
                     .sort(by='pick_dt'))
    df_weekday = df_weekday.rename({'pick_dt': 'pick_day'})
    pocty = np.array([4, 4, 4, 5, 5, 5, 4])
    df_weekday = df_weekday.with_columns(pl.col('pass_count') / pocty)
    graf = px.bar(df_weekday, x='pick_day', y='pass_count', barmode='group', width=750, height=400)
    xtext = ['Pondelok', 'Utorok', 'Streda', 'Štvrtok', 'Piatok', 'Sobota', 'Nedeľa']
    graf.update_layout(xaxis=dict(tickmode='array', tickvals=list(range(1, 8)), title='Deň v týždni',
                       ticktext=xtext, tickangle=0), yaxis=dict(title="Priem. počet cestujúcich"))
    return graf

# v aplikacii budu tie tri grafy - mesacne podla hodin, dni a tyzdenny v jednej 'zalozke'

def view_month_week(doh):
    if doh in ['Podľa dní', 'Podľa hodín']:
        return monthly_plot(doh)
    return week_plot(df)